# Accessibility

This notebook calculates the accessibility of a territory based on the OD matrix previusly calculated.

Accessibility can be regarded as the ability or ease of reaching opportunities while considering their spatial distribution and the cost of traveling via available transport means. Accessibility can be represented as the absolute or relative number of opportunities that can be reached in a certain threshold. This definition follows the equation:

$$
A_i = \sum_{j=0}^n a_j \cdot f(c_{ij})
$$
where:
- $A_i$ is the accessibility of origin $i$
- $a_j$ is the attractiveness of destination $j$
- And $f(c_{ij})$ is a function that depends on the travel cost between origin $i$ and destination $j$.

In [17]:
import pandas as pd
import geopandas as gpd

import utils.analysis as an

In [ ]:
# Import the names of the territories
l_names = pd.read_csv('../data/input/table/mpios_names.txt',header=None)
l_names = list(l_names[0])

# Types of codes
codes_names_list = ['health','sport','education','financial','cultural','parks']

# Threshold in minutes
threshold = 15

In [ ]:
for m in l_names:
    for c in codes_names_list:
        OD = pd.read_csv(f'../data/output/table/OD_dfs_/{m}_OD_df_{c}.txt', dtype={'Origin':str,'Destin':str,'ID_block':str})

        # Para eliminar
        OD = OD[['Origin','Destin','Time']]
        OD = OD.rename(columns={'Time':'Weight'})

        if len(OD) == 0:
            blocks_df = pd.DataFrame(columns=['MANZ_CCNCT','Acc_i','n_people'])
            blocks_df = blocks_df[['MANZ_CCNCT','Acc_i','n_people']]
            blocks_df.to_csv(f'../data/output/table/accessibility_dfs/contour/accessibility_i_contour_{threshold}min_{m}_{c}.txt',index=False)
            
            # Import the territory's blocks
            blocks = gpd.read_file(f'../data/output/shape/blocks/{m}_blocks.shp')
            blocks = blocks[['MANZ_CCNCT','geometry']]
            # Import the population to merge with the blocks
            DANE_data = pd.read_csv('../data/input/table/DANE_2018_personas_manz.txt',low_memory=False)
            pop = DANE_data[['MANZ_CCNCT','poblacion']]
            # Merge the population
            blocks = blocks.merge(pop,on='MANZ_CCNCT',how='left')
            blocks = blocks.fillna(0.0)
            blocks = blocks.rename(columns={'poblacion':'n_people'})
            blocks['Acc_i'] = 0.0
            blocks.to_file(f'../data/output/shape/accessibility/contour/accessibility_i_contour_{threshold}min_{m}_{c}.shp')
            continue

        acc = an.contour_accessibility(OD, threshold, 'Weight', 'absolute')

        # Import the territory's blocks
        blocks = gpd.read_file(f'../data/output/shape/blocks/{m}_blocks.shp')
        blocks = blocks[['MANZ_CCNCT','geometry']]
        # Import the population to merge with the blocks
        DANE_data = pd.read_csv('../data/input/table/DANE_2018_personas_manz.txt',low_memory=False)
        pop = DANE_data[['MANZ_CCNCT','poblacion']]
        # Merge the population
        blocks = blocks.merge(pop,on='MANZ_CCNCT',how='left')
        blocks = blocks.fillna(0.0)
        blocks = blocks.rename(columns={'poblacion':'n_people'})
        # Merge the accessibility
        blocks = blocks.merge(acc,left_on='MANZ_CCNCT', right_on='Origin').drop(['Origin'],axis=1)
        blocks.to_file(f'../data/output/shape/accessibility/contour/accessibility_i_contour_{threshold}min_{m}_{c}.shp')
        # Organize the accessibility information to export as a txt
        blocks_df = blocks.copy()
        blocks_df = blocks_df[['MANZ_CCNCT','Acc_i','n_people']]
        blocks_df = blocks_df.rename(columns={'MANZ_CCNCT':'Origin'})
        blocks_df.to_csv(f'../data/output/table/accessibility_dfs/contour/accessibility_i_contour_{threshold}min_{m}_{c}.txt',index=False)